#pycaretはtrain sampleとtest sampleを予め分けて準備し、5つのステップでtest sampleでの精度を解析する

①training dataとtest dataを読み込む

②狭義のtraining set (training sample全体の7割)を10分割交差検証で比較しハイパーパラメーターのチューニングする

③validation set (training sample全体の3割)も使用して、training sample全体でのモデルを作成する

④training set全体での精度の検証 (internal validation)

⑤予め別に自身で準備したtest sampleで精度を検証する (external validation) →この精度を上げるのが目的

In [ ]:
#パッケージのインストール
!pip install pycaret

In [ ]:
#ライブラリのインポート
import numpy as np
import pandas as pd

In [ ]:
#回帰パッケージのインポート
from pycaret.regression import *

#ここからは実際のデータの読み込み

In [ ]:
# ローカルファイルを直接アップロードする (train data)
from google.colab import files
uploaded = files.upload()

In [ ]:
#train dataの読み込み
train_data = pd.read_csv('train.csv')
train_data.head()

In [ ]:
# ローカルファイルを直接アップロードする (test data)
from google.colab import files
uploaded = files.upload()

In [ ]:
#test dataの読み込み
test_data = pd.read_csv('test.csv')
test_data.head()

#ここからステップ①、pycaretにデータの型を判別させ、training data全体をtrainingのための狭義のtraining sampleとパラメーターを決めるためのvalidation sampleに分割する。

In [ ]:
#train_sizeを指定しない場合は7:3に分割される
#pycaretにデータの型を判別させる
exp1 = setup(train_data, target = 'cancertime', train_size = 0.8,ignore_features = None)

#ここからステップ② 狭義のtraining set (上でtraining sample全体の8割に指定)を10分割交差検証で比較しハイパーパラメーターのチューニングする

In [ ]:
#transformed train sampleでのmodelの比較 (デフォルトは10分割交差検証)
compare_models()

In [ ]:
#lightGBM (Gradient Boosting Machine)を作成
lgbm = create_model('lightgbm')

#ここからハイパーパラメーターをチューニング、pycaretはランダムサーチで最適パラメーターを探索

In [ ]:
#試行回数の指定 n_iter = 10 (時間がかかるので10にしています、50や100に設定してください)
#ここでは'RMSE'を最適なものにしている (他r2などを指定可能)
tuned_lgbm = tune_model(lgbm,  n_iter = 10, fold=10,optimize = 'RMSE')

#ここからステップ③ validation sampleも含めたtraining sample2100例全体でのモデル (final model)を固定する

In [ ]:
#モデルの固定
final_lgbm = finalize_model(tuned_lgbm)

#ここからステップ④　training set全体での精度検証 (internal validation)

このコードは厳密にはtraining sample全体で評価を行っているのではなく、training sampleの一部である (今回ならtraining sampleの2割)　validation sampleのみでの評価を行っている

→training sample全体で行うためには、本プロセスの最後で行っているtest dataでの評価の際に用いるtest dataをtraining dataに置き換えて、test dataに行うのと同様にScoreのCSVファイルを出力する

In [ ]:
#lightgbmモデルのinternal validation
predict_model(final_lgbm);

#ここからステップ⑤　予め別に自身で準備したtest sampleで精度を検証する (external validation)→この精度を上げるのが目的

In [ ]:
#モデルの評価
prediction_lgbm = predict_model(final_lgbm, data = test_data)
prediction_lgbm

#統計ソフトでの解析用に機械学習が出したスコアと正解ラベルをcsvファイルで出力する。

例えば、RMSEは予測値と真値の差を二乗してその平均を平方したものなので、エクセルの予測値と実際の値を用いて計算することもできる

In [ ]:
#モデルの予測出力 (左のフォルダにcsvが出ているので、ファイル右の点3つからダウンロード)
prediction_lgbm.to_csv('prediction_lgbd.csv', columns=['cancertime','prediction_label'])

#ハイパーパラメーターや変数の重要度等を一斉に出力する(一部出力不可のモデルもある)

In [ ]:
evaluate_model (final_lgbm)